# 1. 時系列データを取得し、CSVで保存する

このサンプルでは、以下の方法を中心に解説します。

- スマートフォンのセンサーデータに対して、バイナリデータを数値に変換する信号定義(※)を登録する
- 登録した信号定義を使ってデータを取得する
- 取得した時系列データを`pandas.DataFrame`に変換し、CSVファイルとして保存する

```
 信号定義とは、バイナリ形式でフォーマットされたデータから一部を切り出し、数値や文字列など値データとして読み出す処理を定義したものです。
 この処理はサーバー側で実行されます。変換の種類に関しては、intdash.Conversionの派生クラスのドキュメントを参照してください     
(https://docs.intdash.jp/sdk/python/latest/ja/signals.html#intdash.Conversion)
```

## シナリオ
本シナリオでは、iOSアプリケーション **Visual M2M Motion** からアップロードされた時系列データをダウンロードし、CSV形式でローカルストレージに保存します。

## 事前準備
本シナリオを実施する前に、以下を用意する必要があります。

- 計測用のエッジ
- Visual M2M Motionを使用してアップロードされた計測(スマートフォンのセンサーデータを含む)
- スマートフォンのセンサーデータに紐づく信号定義


### 使用データ
本シナリオでは、以下のデータをサーバー側に準備する必要があります。  

|データ項目|本シナリオで登場するデータ名|
|:---|:---|
|時系列データを登録するエッジ|sdk_edge1|
|信号定義(※)| `sp_ACCX`, `sp_ACCY`, `sp_ACCZ`|

(※)本シナリオでは、スマートフォンのセンサーデータの加速度データを浮動小数点に変換する信号定義を使用します。登録方法については、以下の手順を確認してください。

### 計測データの作成と時系列データのアップロード
**Visual M2M Motion** を使用してデータをアップロードします。 計測を登録後、 **Visual M2M Data Visualizer**  にて計測が新しく生成されたことを確認してください。

### (補足) 本シナリオでのVisual M2M Motion の操作手順
1. iPhoneでVisual M2M Motionを起動し、 `sdk_edge1` としてサインインする。
2. Settings > Sensor > Save to server を有効にする。
3. Main画面に戻り、画面中央の▶をタップする。
4. 計測が開始した画面を確認する
5. ■をタップし、計測を終了する

Visual M2M Data Visualizerの「Stored Data」に計測が表示されていれば完了です。
この後、この計測の時系列データを取得し、CSV形式で保存します。

<img src="https://github.com/aptpod/intdash-py-sample-codes/tree/master/img/img1.png">

### パッケージのimportとクライアントの生成
`intdash.Client` に与える `url` は intdashサーバーの環境情報から、 `edge_token` はログイン用エッジで発行したトークンを指定してください。  
(※ `username` と `password` でのログインも可能ですが、継続して使用する場合はエッジトークンの使用を推奨します)

In [4]:
import pandas as pd

import intdash
from intdash import  timeutils

###  Create Client

client = intdash.Client(
    url = "https://example.intdash.jp",
    edge_token = "your_token",
)

### 信号定義を登録する
本シナリオでは「汎用センサー型」のデータを使用します。このデータはintdash独自の形式であるため、サーバー側でスマートフォンのセンサー型から数値型(今回はFloat型)に変換します。
この変換を行うためには、「信号定義」を登録する必要があります。
スマートフォンのセンサー型から数値に変更するための実行ファイルは、以下を使用してください。

[create-signal-general-sensor.ipynb](../0_create-signals/create-signal-general-sensor.ipynb)  

本サンプルでは、「汎用センサー型」のうち、「加速度」の変換定義のみを登録します。  

### 信号定義が登録されていることを確認する
上記で登録した信号定義を確認します。

In [2]:
signals = client.signals.list(label='sp')

In [3]:
for s in signals:
    print(s.label,  end=', ')

sp_ACCX, sp_ACCY, sp_ACCZ, 

以上で事前準備は終了です。

## 計測に使用したエッジを取得する

In [4]:
edges = client.edges.list(name='sdk_edge1')
sdk_edge1 = edges[0]

In [5]:
sdk_edge1.name

'sdk_edge1'

## (option) 計測を取得する
計測のUUIDを使用して検索したい場合は計測情報を取得します。  
最初の取得時には `list()` を使用して時間指定を行います。  

In [7]:
ms = client.measurements.list(
    edge_uuid=sdk_edge1.uuid,
    start=timeutils.str2timestamp('2020-07-09 00:00:00+09:00'),
    end=timeutils.str2timestamp('2020-07-10 00:00:00+09:00')
)

In [8]:
# Because there is only one measurement associated with `sdk_edge1`, it is specified as follows.
m = ms[0]
print(m)

uuid: db43f826-03a4-44a1-84ca-518fe93c637e
name: 
description: 
edge_uuid: bbe7c17a-6edf-436f-9bc6-c9aad382280c
duration: 0 days 00:00:18.146069
ended: True
basetime: 2020-07-09 08:34:10.832607+00:00
basetime_type: ntp
processed_ratio: 1
protected: True
markers: []
created_at: 2020-07-09 08:34:11.214944+00:00
updated_at: 2020-07-09 08:34:35.376772+00:00


## 時系列データを取得する
時系列データの取得には `client.data_points` エンドポイントを使用します。  
`labels` に、事前に登録している信号定義のラベル名を指定して実行します。

In [9]:
dps = client.data_points.list(
    edge_name='sdk_edge1',
    start=timeutils.str2timestamp('2020-07-09 00:00:00+09:00'),
    end=timeutils.str2timestamp('2020-07-10 00:00:00+09:00'),
    labels=['sp_ACCX', 'sp_ACCY', 'sp_ACCZ']
)

In [10]:
print(dps[0])

time: 2020-07-09T08:34:11.095032000Z
measurement_uuid: db43f826-03a4-44a1-84ca-518fe93c637e
data_type: 11
channel: 1
data_id: sp_ACCY
data_payload: b'\x07sp_ACCY\x7f\x9e\x06\x0c\x92>\xa5?'


## DataPointsのリストをDataFrameに変換する

以下のように指定することで、 `data_id` ごとのカラムを持つDataFrameに変換することができます。

In [19]:
from intdash import data

df = pd.DataFrame( [ {
     'time' : d.time,
       d.data_id : data.Float.from_payload(d.data_payload).value  # convert binary to numbers.
    }
    for d in dps
]).groupby("time").last()

df

,sp_ACCY,sp_ACCZ,sp_ACCX
time,,,
2020-07-09T08:34:11.095032000Z,0.041493,0.217996,0.048094
2020-07-09T08:34:11.115194000Z,0.123338,0.305914,0.038043
2020-07-09T08:34:11.135356000Z,0.046133,0.436974,-0.051388
2020-07-09T08:34:11.155517000Z,-0.034988,0.233272,0.102698
2020-07-09T08:34:11.175679000Z,-0.055907,-0.164569,0.047395
...,...,...,...
2020-07-09T08:34:14.361262000Z,0.506122,0.231640,-1.372483
2020-07-09T08:34:14.381424000Z,0.354834,-0.354698,-1.570656
2020-07-09T08:34:14.401586000Z,-0.216771,0.578869,-0.578583


## CSV形式で保存する
DataFrameの機能を使い、CSVファイルに変換します。

In [20]:
df.to_csv('./sdk_sample.csv')

このように保存されます。

<img src="https://github.com/aptpod/intdash-py-sample-codes/tree/master/img/img2.png">